# Plotting notebook for PiC_UVnudge runs
## Set up
### Packages

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import pandas as pd
import scipy
from scipy import stats
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.mathtext import _mathtext as mathtext
import matplotlib.ticker as mticker
from matplotlib import font_manager
from matplotlib import gridspec, animation
import matplotlib.path as mpath
import matplotlib.colors as colors
import matplotlib.dates as mdates
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import warnings
warnings.simplefilter('ignore', UserWarning)
warnings.filterwarnings('ignore')
import datetime as dt
from datetime import timedelta
from cmcrameri import cm
import jinja2
from Plotting_functions import wvl2wvn, wvn2wvl, p2z, z2p, t_test_two_means, Wilks_pcrit, CustomCmap, draw_circle, CalcStatSig
import cftime
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
from functools import partial
from collections import defaultdict

In [2]:
font_path = '/glade/work/glydia/conda-envs/cenv/fonts/Helvetica.ttc'  # Your font path goes here
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Helvetica'

### Filepaths and name variables

In [3]:
## Plot types to make - CHANGE
# 0: Weighted spatial mean or Arctic sea ice area
# 1: Spatial maximum (AMOC)
# 2: Leave alone (doing spatial map or sea ice concentration)
plots = {
    'amoc': [False, 1], # Plot AMOC
    'toa': [False, 0],  # Plot net TOA imbalance
    'map': [False, 2],  # Plot spatial average map
    'ts': [False, 0],   # Plot average Arctic/Global temperature
    'sia': [False, 0],  # Plot sea ice area for one month
    'mtrd': [True, 0],  # Plot monthly and annual trends
    'strd': [False, 2], # Plot spatial trends
    'zon': [False, 3], # Plot zonal data
    'ztrd': [False, 3] # Plot zonal trend data
}

## Categorical plot type - DO NOT CHANGE
plot_types = {
    'spatial': plots['map'][0] or plots['strd'][0],
    'line': plots['amoc'][0] or plots['toa'][0] or plots['ts'][0] or plots['sia'][0],
    'mtrd': plots['mtrd'][0],
    'zonal': plots['zon'][0] or plots['ztrd'][0]
}

# Spatial & time domain - CHANGE s_domain & t_domain only
s_domain = 1 # 0: Global, 1: Arctic, 2: Mid-latitudies
s_domain = 0 if (plots['toa'][0] or plots['amoc'][0]) else s_domain # Make sure TOA is global domain
a_domain = plot_types['spatial'] or plot_types['zonal'] # True: 50-90, False: 70-90
t_domain = 1980 # start year

## Time averaging type - CHANGE
time_avg = 4   # 0: Monthly, 1: Yearly, 2: Seasonal, 3: All data, 4: Timeseries

## Ensemble mean or All members - CHANGE
ens_type = 0   # 0: All_members, 1: Mean

# Variables - CHANGE
comp = 'ice'
freq = 0 # 0: monthly, 1: daily
var_ind = 0

# DO NOT CHANGE
var_list = {'atm': ['TREFHT','PSL','RESTOM','Z3'],
            'ice': ['aice'],
            'ocn': ['MOC']}
var_ext = {0: '', 1: '_d'}
var = var_list[comp][var_ind]+var_ext[freq]

# CHANGE
type_onemonth = 9 # Selects single month to plot timeseries for (time_avg = 4), if doing all months, equals 0
plot_levels = [300,500, 850,925] # Selects plot levels for vlev type plots

In [4]:
## Test numbers - DO NOT CHANGE
tst_nums = np.arange(1,4)

## Test names - True means plot, False means don't plot
# CHANGE
ds_plot_list = {
    'PiC_UVnudge': False,
    'PiC_UVnudge_LM': False,
    'PiC_UVnudge_MM': False,
    'PiC_UVnudge_2006': True,
    'PiC_UVnudge_LM2006': True,
    'PiC_UVnudge_MM2006': True,
    'PiC_UVnudge_2006_2000': False,
    'PiC_UVnudgenew': False,
    'PiC_UVnudge_1988': False,
    'PiC_UVnudge_global': False,
    'PiC_UVnudge_3day': False,
    'LENS2 piControl': True
}

## Filepaths - DO NOT CHANGE
path_to_data = '/glade/work/glydia/Arctic_controls_processed_data/plotting_data/'
path_to_graphs = '/glade/u/home/glydia/PiC_UVnudge_graphs/'

# Conditions - DO NOT CHANGE
vert_lev = {'atm': [False,False,False,True],
            'ice': [False],
            'ocn': [False]}
file_bool = not vert_lev[comp][var_ind] and freq == 0

In [5]:
########################## DO NOT CHANGE ANYTHING BELOW THIS LINE #############################

In [6]:
%%time
    
## Select plot type
time_str_list = {0: 'month', 1: 'year', 2: 'season', 3: 'all', 4: 'timeseries'}
time_outstr = time_str_list[time_avg]

## Select ensemble type
ens_str_list = {0: 'All_members', 1: 'Mean'}
ens_str = ens_str_list[ens_type]

## Select time and spatial domain strings
sd_str_list = {0: 'Global', 1: 'Arctic', 2: 'MidLat'}
sd_str = sd_str_list[s_domain]
td_str = str(t_domain)

mon_str = np.array(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
month_str = np.array(['January','February','March','April','May','June','July','August','September','October','November','December'])
seas_str = np.array(['MAM','JJA','SON','DJF'])

CPU times: user 30 µs, sys: 0 ns, total: 30 µs
Wall time: 32.9 µs


In [7]:
## Set up properties of each dataset and whether it will be plotted or not
# O: LENS ensemble
# 1: PiC_UVnudge ensemble
# 2: PiC_UVnudge single run
# 3: observations
# attribute structure: [use dataset[0], dataset typ[1], line color[2], line style[3], zorder[4], note[5](optional)]
ds_names = {
    'ERA5': [not (plots['amoc'][0] or plots['toa'][0]), 3, 'black', '-', 2],
    'GISTEMP': [plots['ts'][0] or (plots['mtrd'][0] and var == 'TREFHT'), 3, 'black', '--', 2],
    'NSIDC': [plots['sia'][0] or (plots['mtrd'][0] and var == 'aice'), 3, 'black', '--', 2],
    'LENS2 piControl': [True, 0, 'tab:blue', '-', 1],
    'PiC_UVnudge':  [None, 1, 'purple', '-', 3, 'p'],
    'PiC_UVnudge_LM': [None, 1, 'darkblue', '-', 3, 'lm'],
    'PiC_UVnudge_MM': [None, 1, 'darkred', '-', 3, 'mm'],
    'PiC_UVnudge_2006': [None, 1, 'm', '-', 4, 'p06'],
    'PiC_UVnudge_LM2006': [None, 1, 'blue', '-', 4, 'lm06'],
    'PiC_UVnudge_MM2006': [None, 1, 'red', '-', 4, 'mm06'],
    'PiC_UVnudge_2006_2000': [None, 2, 'mediumorchid', '-', 3, 'p0600'],
    'PiC_UVnudgenew': [None, 2, 'magenta', '-', 3, 'pnp'],
    'PiC_UVnudge_1988': [None, 2, 'darkorange', '-', 3, 'p88'],
    'PiC_UVnudge_global': [None, 2, 'gold', '-', 3, 'pg'],
    'PiC_UVnudge_3day': [None, 2, 'olivedrab', '-', 3, 'p3']
}

ds_paper_names = {
    'LENS2 piControl': 'PI-control',
    'PiC_UVnudge_2006': 'PInudge',
    'PiC_UVnudge_LM2006': 'PInudge-lessmelt',
    'PiC_UVnudge_MM2006': 'PInudge-moremelt',
    'ERA5': 'OBS (ERA5)',
    'GISTEMP': 'OBS (GISTEMP)',
    'NSIDC': 'OBS (NSIDC)',
    'PiC_UVnudge': 'PiC_UVnudge',
    'PiC_UVnudge_LM': 'PiC_UVnudge_LM',
    'PiC_UVnudge_MM': 'PiC_UVnudge_MM',
    'PiC_UVnudgenew': 'PiC_UVnudgenew',
    'PiC_UVnudge_2006_2000': 'PiC_UVnudge_2006_2000',
    'PiC_UVnudge_1988': 'PiC_UVnudge_1988',
    'PiC_UVnudge_global': 'PiC_UVnudge_global',
    'PiC_UVnudge_3day': 'PiC_UVnudge_3day'
}

# Update PiC_UVnudge 'use dataset' values with T/F from ds_plot_list
for dsname, use in ds_plot_list.items():
    ds_names[dsname][0] = use

In [8]:
## Determine note type based on model runs plotted
note = ''
driftnote = ''

for dsname, attrs in ds_names.items():
    # Only pick datasets that will plotted and that have a code
    if attrs[0] and len(attrs) == 6:
        # Each PiC_UVnudge run has its own code, if the run will be plotted, it's code will be added to the note
        # If dataset is drift
        if ' drift' in dsname:
            driftnote = driftnote+attrs[5]+'-'
        # Else dataset is not drift
        else:
             note = note+attrs[5]+'-'

note = 'era5' if note == '' else note[:-1]
driftnote = driftnote if driftnote == '' else driftnote[:-1]

### Custom functions

In [9]:
def LoadData(plot_type, varname, tavg, plot_level=None):
    # Create file name
    filename = plot_type+'.'+varname+'.'+sd_str+'.'+td_str+'.'+tavg+'.'+ens_str+'.nc'
    filepath = path_to_data+filename
    print(filename)
    
    data = xr.open_dataset(filepath)
    
    return data

In [10]:
def SubCmap(orgcmap, levels, type, color):
    # Type is middle, end or beginning
    # Make cmap values swapped out
    # color is color to sub in
    lev_len = len(levels)+1
    cmap_samp = orgcmap.resampled(lev_len)
    cmap_list = cmap_samp(np.linspace(0,1,lev_len))

    # Swap out first two
    if type == 'beg':
        lev_ind = 0
        cmap_list[lev_ind:(lev_ind+1), :] = color[0]
        cmap_list[(lev_ind+1):(lev_ind+2), :] = color[1]
        final_cmap, _ = CustomCmap(levels, cmap_list, 
                           [color[0], orgcmap.get_over()], True)

    # Swap out middle two
    elif type == 'mid':
        lev_ind = int(lev_len/2)
        cmap_list[(lev_ind-1):(lev_ind+1), :] = color
        final_cmap, _ = CustomCmap(levels, cmap_list, 
                           [orgcmap.get_under(), orgcmap.get_over()], True)

    # Swap out last
    else:
        lev_ind = -1
        cmap_list[lev_ind:, :] = color
        final_cmap, _ = CustomCmap(levels, cmap_list, 
                           [orgcmap.get_under(), color], True)

    return final_cmap

In [11]:
def TrendLine(da, period):
    da = da.dropna(dim=period)
    # Calculate regression coefficients
    da_reg_coef = da.polyfit(dim=period, deg=1)

    # Calculate trend line
    da_yreg = (da_reg_coef.loc[dict(degree=1)]*da[period])+da_reg_coef.loc[dict(degree=0)]

    return da_yreg['polyfit_coefficients']

In [12]:
def AxisLabels(ax, title_str, land):
    if land:
        ax.add_feature(cfeature.LAND, zorder=3)
    ax.coastlines(zorder=4)
    ax.set_extent(extent, ccrs.PlateCarree())
    ax.set_title(title_str, fontsize = 10)
    draw_circle(ax, draw_circ=(s_domain == 1), draw_major=False)
    return None

In [13]:
def SaveFig(fig, plot_type, tavg, varname, note=None, plot_level=None):
    # File format is:
    # var.ensemble_type.spatialdomain.[Zplot_level].plot_type.timeaveraging.timedomain.[note].png
    level_str = '' if plot_level == None else 'Z'+str(plot_level)+'.'
    note_str = '' if note == None else note+'.'
    
    fig.savefig(path_to_graphs+varname+'.'+ens_str+'.'+sd_str+'.'+level_str+plot_type+'.'+tavg+'.'+td_str+'.'+note_str+'pdf', bbox_inches='tight')
    return None

## Month trend plots
### Set up

In [14]:
if plot_types['mtrd']:
    graph_type_str = 'Linear.Trend'

    # Graph labels and variables
    xtick_loc = np.arange(1,13)
    xtick_lbl = mon_str
    xlim = [0.9,12.1]
    xlabel= 'Month'
    date_str = 'Month'
    period = 'month'
    dim_avg = 'time.month'
        

    ylabel = {'aice': 'Sea ice extent trend (million km$^2$/decade)',
             'TREFHT': '2m air temperature trend (K/decade)'}
    ylabelrat = {'aice': 'Percent of OBS sea ice extent\ntrend explained (%)',
              'TREFHT': 'Percent of OBS 2m air temperature\ntrend explained (%)'}
    yticks = {'aice': np.arange(-1.2, 0.41,0.2),
              'TREFHT': np.arange(-1.0,1.51,0.5)}
    yticksrat = {'aice': np.arange(-10,51,10),
                 'TREFHT': np.arange(-20,51,10)}
    ylim = {'aice': [-1.3,0.4],
            'TREFHT': [-1,1.5]}
    ylimrat = {'aice': [-15,50],
            'TREFHT': [-20,60]}
    bbox_loc = {'aice':(0.01, 0.39),
              'TREFHT': (0.32, 0.39)}
    bbox_loc_rat = {'aice':(0.25, 0.99),
                    'TREFHT': (0.45, 0.99)}

### Data loading

In [15]:
%%time

if plot_types['mtrd']:

    # Load monthly trends
    ds_mon_trd = LoadData(graph_type_str, var, 'month')

    # Load annual trends
    ds_ann_trd = LoadData(graph_type_str, var, 'year')

Linear.Trend.aice.Arctic.1980.month.All_members.nc
Linear.Trend.aice.Arctic.1980.year.All_members.nc
CPU times: user 988 ms, sys: 188 ms, total: 1.18 s
Wall time: 1.22 s


### Make plots

In [16]:
%%time

if plot_types['mtrd']:
    ## Make standard monthly trend plots
    # Set up 
    fig = plt.figure(figsize=(4,3.5), layout='constrained')
    gs = fig.add_gridspec(1, 2,  width_ratios=(6, 1), wspace=0)
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])

    # Plotting
    ax1.axhline(0,color='k',lw=0.5,zorder=0)
    ax2.axhline(0,color='k',lw=0.5,zorder=0)
    
    for dsname, attrs in ds_names.items():
        if attrs[0]:
            da_mon = ds_mon_trd[dsname]
            da_ann = ds_ann_trd[dsname]

            # LENS ensemble
            if attrs[1] == 0:
                # Extract mean & min and max of envelople
                da_mon_mean = ds_mon_trd[dsname+' mean']
                da_mon_min = ds_mon_trd[dsname+' min']
                da_mon_max = ds_mon_trd[dsname+' max']

                da_ann_mean = ds_ann_trd[dsname+' mean']
                da_ann_min = ds_ann_trd[dsname+' min']
                da_ann_max = ds_ann_trd[dsname+' max']
                
                # Plot envelope
                ax1.fill_between(da_mon_min[period].values, da_mon_min.values,
                    da_mon_max.values, color=attrs[2], alpha=0.1,
                    ec=None, zorder=attrs[4],label=ds_paper_names[dsname])
                ax2.axhspan(ymin=da_ann_min.values, ymax=da_ann_max.values,
                   xmin=0.45, xmax=0.55, alpha=0.1, color=attrs[2],ec=None,zorder=attrs[4])
                
                # Plot mean
                # da_mon_mean.plot(
                #     ax=ax1, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4],label=ds_paper_names[dsname]+' mean')
                # ax2.axhline(da_ann_mean.values,xmin=0.45,xmax=0.55, ls=attrs[3],
                #     zorder=attrs[4], color=attrs[2])

            # PiC_UVnudge ensemble
            elif attrs[1] == 1:
                index = dict(ensemble_member=1)

                # Plot first ensemble member
                da_mon.loc[index].plot(
                    ax=ax1, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname])
                ax2.axhline(da_ann.loc[index].values, xmin=0.45, xmax=0.55, zorder=attrs[4], 
                            ls=attrs[3], color=attrs[2]) 

                # Plot other ensemble members
                for i in range(2,4):
                    index['ensemble_member'] = i
            
                    da_mon.loc[index].plot(
                        ax=ax1, color=attrs[2], x=period, zorder=attrs[4], ls=attrs[3])
                    ax2.axhline(da_ann.loc[index].values, xmin=0.45, xmax=0.55,
                           zorder=attrs[4], color=attrs[2], ls=attrs[3])
                    
            # PiC_UVnudge single run & Observations
            elif attrs[1] >= 2:
                # Plot monthly & annual trends
                da_mon.plot(
                    ax=ax1, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname])
                ax2.axhline(da_ann.values, xmin=0.45, xmax=0.55,
                    color=attrs[2], ls=attrs[3], zorder=attrs[4])
    
    # Formatting
    ax1.set_title('')
    ax1.set_xlim(xlim)
    ax1.set_ylim(ylim[var])
    ax1.set_ylabel(ylabel[var])
    ax1.set_xticks(ticks=xtick_loc, labels=xtick_lbl, rotation='vertical')
    ax1.set_xlabel(xlabel)
    ax1.legend(loc='upper left', framealpha=0, bbox_to_anchor=bbox_loc[var], fontsize=8)
    ax1.grid(alpha=0.3)
    
    ax2.set_ylim(ylim[var])
    ax2.set_yticks([])
    ax2.set_xticks(ticks=[0.5],labels=['ANNUAL'],rotation='vertical')
    ax2.grid(alpha=0.3)
    for tick in yticks[var]:
        ax2.axhline(tick,color='tab:gray',zorder=0,lw=0.5,alpha=0.3)

    SaveFig(fig, graph_type_str, 'monyr', var, note)

    plt.close(fig)

CPU times: user 389 ms, sys: 9.11 ms, total: 399 ms
Wall time: 586 ms


In [17]:
%%time

if plot_types['mtrd']:
    # Make ratio of exp/obs trend explained plots
    # Set up 
    fig = plt.figure(figsize=(4,3.5), layout='constrained')
    gs = fig.add_gridspec(1, 2,  width_ratios=(6, 1), wspace=0)
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])

    # Plotting
    ax1.axhline(0,color='k',lw=0.5,zorder=0)
    ax2.axhline(0,color='k',lw=0.5,zorder=0)

    da_mon_era = ds_mon_trd['ERA5']
    da_ann_era = ds_ann_trd['ERA5']
    
    for dsname, attrs in ds_names.items():
        if attrs[0]:
            da_mon = ds_mon_trd[dsname]
            da_ann = ds_ann_trd[dsname]

            # PiC_UVnudge ensemble
            if attrs[1] == 1:
                index = dict(ensemble_member=1)

                # Plot first ensemble member
                ((da_mon.loc[index]/da_mon_era)*100).plot(
                    ax=ax1, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname]+'/\n'+ds_paper_names['ERA5'])
                ax2.axhline(((da_ann.loc[index]/da_ann_era)*100).values, xmin=0.45, xmax=0.55, zorder=attrs[4], 
                            ls=attrs[3], color=attrs[2]) 

                # Plot other ensemble members
                for i in range(2,4):
                    index['ensemble_member'] = i
            
                    ((da_mon.loc[index]/da_mon_era)*100).plot(
                        ax=ax1, color=attrs[2], x=period, zorder=attrs[4], ls=attrs[3])
                    ax2.axhline(((da_ann.loc[index]/da_ann_era)*100).values, xmin=0.45, xmax=0.55,
                           zorder=attrs[4], color=attrs[2], ls=attrs[3])
                    
            # PiC_UVnudge single run & Observations
            elif attrs[1] == 2:
                # Plot monthly & annual trends
                ((da_mon/da_mon_era)*100).plot(
                    ax=ax1, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname]+'/\n'+ds_paper_names['ERA5'])
                ax2.axhline(((da_ann/da_ann_era)*100).values, xmin=0.45, xmax=0.55,
                    color=attrs[2], ls=attrs[3], zorder=attrs[4])
    
    # Formatting
    ax1.set_title('')
    ax1.set_xlim(xlim)
    ax1.set_ylim(ylimrat[var])
    ax1.set_ylabel(ylabelrat[var])
    ax1.set_xticks(ticks=xtick_loc, labels=xtick_lbl, rotation='vertical')
    ax1.set_xlabel(xlabel)
    ax1.legend(loc='upper left', framealpha=0, bbox_to_anchor=bbox_loc_rat[var], fontsize=8)
    ax1.grid(alpha=0.3)
    
    ax2.set_ylim(ylimrat[var])
    ax2.set_yticks([])
    ax2.set_xticks(ticks=[0.5],labels=['ANNUAL'],rotation='vertical')
    ax2.grid(alpha=0.3)
    for tick in yticksrat[var]:
        ax2.axhline(tick,color='tab:gray',zorder=0,lw=0.5,alpha=0.3)

    SaveFig(fig, graph_type_str+'.ratio', 'monyr', var, note)

    plt.close(fig)

CPU times: user 197 ms, sys: 7.12 ms, total: 204 ms
Wall time: 323 ms


In [18]:
%%time

if plot_types['mtrd']:
    ## Make standard monthly trend plots + ratio of exp/obs trend explained plots in one figure
    # Set up 
    fig = plt.figure(figsize=(8,3.5), layout='constrained')
    gs = fig.add_gridspec(1, 5,  width_ratios=(6, 1,0.5,6, 1), wspace=0)
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])
    ax3 = fig.add_subplot(gs[3])
    ax4 = fig.add_subplot(gs[4])

    let_list = ['(a)', '(b)']

    # Plotting
    ax1.axhline(0,color='k',lw=0.5,zorder=0)
    ax2.axhline(0,color='k',lw=0.5,zorder=0)
    ax3.axhline(0,color='k',lw=0.5,zorder=0)
    ax4.axhline(0,color='k',lw=0.5,zorder=0)

    da_mon_era = ds_mon_trd['ERA5']
    da_ann_era = ds_ann_trd['ERA5']
    
    for dsname, attrs in ds_names.items():
        if attrs[0]:
            da_mon = ds_mon_trd[dsname]
            da_ann = ds_ann_trd[dsname]

            # LENS ensemble
            if attrs[1] == 0:
                # Extract mean & min and max of envelople
                da_mon_min = ds_mon_trd[dsname+' min']
                da_mon_max = ds_mon_trd[dsname+' max']

                da_ann_min = ds_ann_trd[dsname+' min']
                da_ann_max = ds_ann_trd[dsname+' max']
                
                # Plot envelope
                ax1.fill_between(da_mon_min[period].values, da_mon_min.values,
                    da_mon_max.values, color=attrs[2], alpha=0.1,
                    ec=None, zorder=attrs[4],label=ds_paper_names[dsname])
                ax2.axhspan(ymin=da_ann_min.values, ymax=da_ann_max.values,
                   xmin=0.45, xmax=0.55, alpha=0.1, color=attrs[2],ec=None,zorder=attrs[4])

            # PiC_UVnudge ensemble
            elif attrs[1] == 1:
                index = dict(ensemble_member=1)

                # Plot first ensemble member
                da_mon.loc[index].plot(
                    ax=ax1, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname])
                ((da_mon.loc[index]/da_mon_era)*100).plot(
                    ax=ax3, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname]+'/'+ds_paper_names['ERA5'])
                
                ax2.axhline(da_ann.loc[index].values, xmin=0.45, xmax=0.55, zorder=attrs[4], 
                            ls=attrs[3], color=attrs[2]) 
                ax4.axhline(((da_ann.loc[index]/da_ann_era)*100).values, xmin=0.45, xmax=0.55, zorder=attrs[4], 
                            ls=attrs[3], color=attrs[2]) 

                # Plot other ensemble members
                for i in range(2,4):
                    index['ensemble_member'] = i
            
                    da_mon.loc[index].plot(
                        ax=ax1, color=attrs[2], x=period, zorder=attrs[4], ls=attrs[3])
                    ((da_mon.loc[index]/da_mon_era)*100).plot(
                        ax=ax3, color=attrs[2], x=period, zorder=attrs[4], ls=attrs[3])
                    
                    ax2.axhline(da_ann.loc[index].values, xmin=0.45, xmax=0.55,
                           zorder=attrs[4], color=attrs[2], ls=attrs[3])
                    ax4.axhline(((da_ann.loc[index]/da_ann_era)*100).values, xmin=0.45, xmax=0.55,
                           zorder=attrs[4], color=attrs[2], ls=attrs[3])
         
            # PiC_UVnudge single run & Observations
            elif attrs[1] >= 2:
                # Plot monthly & annual trends
                da_mon.plot(
                    ax=ax1, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname])
                ax2.axhline(da_ann.values, xmin=0.45, xmax=0.55,
                    color=attrs[2], ls=attrs[3], zorder=attrs[4])

                if attrs[1] == 2:
                    ((da_mon/da_mon_era)*100).plot(
                        ax=ax3, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname]+'/'+ds_paper_names['ERA5'])
                    ax4.axhline(((da_ann/da_ann_era)*100).values, xmin=0.45, xmax=0.55,
                        color=attrs[2], ls=attrs[3], zorder=attrs[4])
                
    
    # Formatting
    ax1.set_title('')
    ax1.set_xlim(xlim)
    ax1.set_ylim(ylim[var])
    ax1.set_ylabel(ylabel[var])
    ax1.set_xticks(ticks=xtick_loc, labels=xtick_lbl, rotation='vertical')
    ax1.set_xlabel(xlabel)
    ax1.legend(loc='upper left', framealpha=0, bbox_to_anchor=bbox_loc[var], fontsize=8)
    ax1.grid(alpha=0.3)
    ax1.annotate(let_list[0],(0.15,0.93),xytext=(-10,0),ha='right',va='center',
                    size=12,xycoords='axes fraction',textcoords='offset points')
    
    ax2.set_ylim(ylim[var])
    ax2.set_yticks([])
    ax2.set_xticks(ticks=[0.5],labels=['ANNUAL'],rotation='vertical')
    ax2.grid(alpha=0.3)
    for tick in yticks[var]:
        ax2.axhline(tick,color='tab:gray',zorder=0,lw=0.5,alpha=0.3)

    ax3.set_title('')
    ax3.set_xlim(xlim)
    ax3.set_ylim(ylimrat[var])
    ax3.set_ylabel(ylabelrat[var])
    ax3.set_xticks(ticks=xtick_loc, labels=xtick_lbl, rotation='vertical')
    ax3.set_xlabel(xlabel)
    ax3.legend(loc='upper left',framealpha=0, bbox_to_anchor=bbox_loc_rat[var], fontsize=8)
    ax3.grid(alpha=0.3)
    ax3.annotate(let_list[1],(0.15,0.93),xytext=(-10,0),ha='right',va='center',
                    size=12,xycoords='axes fraction',textcoords='offset points')
    
    ax4.set_ylim(ylimrat[var])
    ax4.set_yticks([])
    ax4.set_xticks(ticks=[0.5],labels=['ANNUAL'],rotation='vertical')
    ax4.grid(alpha=0.3)
    for tick in yticksrat[var]:
        ax4.axhline(tick,color='tab:gray',zorder=0,lw=0.5,alpha=0.3)

    SaveFig(fig, graph_type_str, 'monyr-ratio', var, note)

    plt.close(fig)

CPU times: user 503 ms, sys: 3.94 ms, total: 507 ms
Wall time: 514 ms


## Line plots
### Set up

In [19]:
if plot_types['line']:
    graph_type_str = 'Linear'

    ylabelabs = {'aice': 'Sea ice extent (million km$^2$)',
                 'TREFHT': '2m air temperature (K)',
                 'RESTOM': 'Net TOA Imbalance (W/m$^{2}$)',
                 'MOC': 'Atlantic Meridional Overturning Circulation (Sv)'}
    ylabelanom = {'TREFHT': '2m air temperature anomaly (K)'}
    ylabeldtrd = {'TREFHT': 'Detrended 2m air temperature anomaly (K)'}
    ylabelensp = {'TREFHT': '2m air temperature ensemble spread (K)'}
    ylabel1m = {'aice': ' sea ice extent (million km$^2$)',
                'TREFHT': ' 2m air temperature (K)'}
    ylimabs = {'aice': {3: [13.5,16.5],
                        9: [3,10]},
               'TREFHT': [254.5,265]}

    wdth = {1950: 6,
           1980: 6}

    legend_loc = {'aice':{1950: 'best',
                        1980: 'best'},
                 'MOC': {1950: 'best',
                        1980: (0.5,0.75)},
                 'RESTOM':{1950: 'best',
                        1980: 'best'},
                 'TREFHT':{1950: 'best',
                        1980: 'best'}}

    double_plot = (type_onemonth == 39 and var == 'aice')

    # Prepare ylim
    if var in ylimabs and t_domain == 1980:
        if var == 'aice' and type_onemonth < 13:
            ylim = ylimabs[var][type_onemonth]
        elif var == 'TREFHT':
            ylim = ylimabs[var]
        else:
            ylim = False
    else:
        ylim = False
    
    # Prepare x ticks
    # Timeseries
    if time_avg == 4:
        xtick_loc = np.arange(t_domain,2024,5)
        xtick_loc_min = np.arange(t_domain,2024,1)
        xtick_lbl = np.arange(t_domain,2024,5)
        xlim = [t_domain,2024]
        period ='time'
        xlabel = 'Year'
        xlabelnud = 'Years Nudged'

        if type_onemonth < 13:
            date_str = mon_str[type_onemonth-1]
            ylabelabs[var] = month_str[type_onemonth-1]+ylabel1m[var]
        else:
            date_str = mon_str[3-1]+'-'+mon_str[9-1]
            ylabelabs[var] = {3: month_str[3-1]+ylabel1m[var],
                             9:month_str[9-1]+ylabel1m[var]}
        
        
    # Yearly
    elif time_avg == 1:
        xtick_loc = np.arange(t_domain,2024,5)
        xtick_loc_min = np.arange(t_domain,2024,1)
        xtick_lbl = np.arange(t_domain,2024,5)
        xlim = [t_domain,2023]
        date_str = 'timeseries_yr'
        period='year'
        xlabel= 'Year'
        dim_avg = 'time.year'
        xlabelnud = 'Years Nudged'

### Data loading

In [20]:
%%time

if plot_types['line']:

    ## Absolute (only one that will be used for SIA-one month, TOA, AMOC, drift)
    # Yearly
    if time_avg == 1:
        ds_abs = LoadData(graph_type_str+'.abs', var, period)

        # Number of nudged years
        if t_domain == 1950 and s_domain == 1:
            ds_nudyr = LoadData(graph_type_str+'.nudyr', var, period)

            ds_nudyr_names = {
                'PiC_UVnudge': '1st cycle',
                'PiC_UVnudge_2006': '2nd cycle',
                'PiC_UVnudge_2006_2000': '3rd cycle',
                'PiC_UVnudge_LM': '1st cycle',
                'PiC_UVnudge_LM2006': '2nd cycle',
                'PiC_UVnudge_MM': '1st cycle',
                'PiC_UVnudge_MM2006': '2nd cycle'
            }

        # For temperature plots only
        if plots['ts'][0]:
            # Absolute correlation coefficients
            ds_abs_r = LoadData(graph_type_str+'.absR', var, period)

            # Anomalies
            ds_anom = LoadData(graph_type_str+'.anom', var, period)    
        
            # Anomalies correlation coefficients
            ds_anom_r = LoadData(graph_type_str+'.anomR', var, period)    

            # Detrended anomalies
            ds_dtrd = LoadData(graph_type_str+'.anomdtrd', var, period)    

            # Detrended anomalies correlation coefficients
            ds_dtrd_r = LoadData(graph_type_str+'.anomdtrdR', var, period)    

            if ens_type:
                # Ensemble spread
                ds_enspd = LoadData(graph_type_str+'.espd', var, period)

    # Timeseries of one month of sea ice area data
    elif time_avg == 4:
        # If only plotting one month
        if type_onemonth < 13:
            ds_abs = LoadData(graph_type_str+'.abs', var, mon_str[type_onemonth-1])

            # Absolute correlation coefficients
            ds_abs_r = LoadData(graph_type_str+'.absR', var, mon_str[type_onemonth-1])

            # Number of nudged years
            if t_domain == 1950:
                ds_nudyr = LoadData(graph_type_str+'.nudyr', var, mon_str[type_onemonth-1])
                
        # Elif plotting Mar & Sept on 1 figure
        elif type_onemonth == 39:
            # Load & combine absolute data
            ds_mar = LoadData(graph_type_str+'.abs', var, mon_str[3-1])
            ds_sep = LoadData(graph_type_str+'.abs', var, mon_str[9-1])
            ds_abs = xr.concat([ds_mar,ds_sep],pd.Index([3,9],name='month'))

            # Load & combine absolute correlation coefficients
            ds_mar_r = LoadData(graph_type_str+'.absR', var, mon_str[3-1])
            ds_sep_r = LoadData(graph_type_str+'.absR', var, mon_str[9-1])
            ds_abs_r = xr.concat([ds_mar_r,ds_sep_r],pd.Index([3,9],name='month'))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


In [21]:
%%time

if plot_types['line']:
    if time_avg == 1 and t_domain == 1950 and s_domain == 1:
        # Calculate linear trend for nudge years - ensemble means
        trdline_list = []
        for dsname, attrs in ds_names.items():
            if attrs[0] and ('PiC_UVnudge' in dsname):
                da = ds_nudyr[dsname]
                da = da.mean('ensemble_member') if (attrs[1] == 1 and not ens_type) else da
                da_trdline = TrendLine(da, period)
                trdline_list.append(da_trdline.rename(dsname))

        ds_trdline = xr.merge(trdline_list)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


### Make absolute plots

In [22]:
%%time

if plot_types['line'] and not double_plot:
    # Setup figure
    fig,ax = plt.subplots(1,1, layout='constrained')
    fig.set_size_inches(wdth[t_domain],3.5)

    for dsname, attrs in ds_names.items():
        if attrs[0] and dsname != 'GISTEMP' and (' drift' not in dsname):
            da_abs = ds_abs[dsname]

            # LENS ensemble
            if attrs[1] == 0:
                # Extract mean & min and max of envelope
                da_abs_mean = ds_abs[dsname+' mean']
                da_abs_min = ds_abs[dsname+' min']
                da_abs_max = ds_abs[dsname+' max']
                
                # Plot envelope
                ax.fill_between(da_abs_min[period].values, da_abs_min.values,
                    da_abs_max.values, color=attrs[2], alpha=0.1,
                    ec=None,label=ds_paper_names[dsname], zorder=attrs[4])
                
                # Plot mean
                # da_abs_mean.plot(
                #     ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4],label=ds_paper_names[dsname]+' mean')

            # PiC_UVnudge ensemble
            elif attrs[1] == 1 and not ens_type:
                index = dict(ensemble_member=1)
                addon = ' ($\it{r}^2$='+'{:.2f}; {:.2f})'.format(ds_abs_r[dsname].values**2, ds_dtrd_r[dsname].values**2) if plots['ts'][0] else ''
                addon = ' ($\it{r}^2$='+'{:.2f})'.format(ds_abs_r[dsname].values**2) if plots['sia'][0] else addon
                    
                # Plot first ensemble member
                da_abs.loc[index].plot(
                    ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname]+addon)

                # Plot other ensemble members
                for i in range(2,4):
                    index['ensemble_member'] = i
            
                    da_abs.loc[index].plot(
                        ax=ax, color=attrs[2], x=period, zorder=attrs[4], ls=attrs[3])
                    
            # PiC_UVnudge single run & Observations (& PiC_UVensemble means)
            elif attrs[1] >= 2 or (ens_type and attrs[1] == 1):
                # addon = ' ($\it{r}^2$='+'{:.2f}; {:.2f})'.format(ds_abs_r[dsname].values**2, ds_dtrd_r[dsname].values**2) if ((plots['ts'][0]) and attrs[1] != 3) else ''
                # addon = ' ($\it{r}^2$='+'{:.2f})'.format(ds_abs_r[dsname].values**2) if ((plots['sia'][0]) and attrs[1] != 3) else addon
                addon=''
               
                # Plot absolute value
                da_abs.plot(
                    ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname]+addon)
            

    # Formatting
    ax.set_xlabel(xlabel)
    if time_avg == 1:
        ax.set_xlim(xlim)
        ax.set_xticks(ticks=xtick_loc,labels=xtick_lbl)
        ax.set_xticks(ticks=xtick_loc_min,minor=True)
    elif time_avg == 4:
        ax.xaxis.set_major_locator(mdates.YearLocator(5,month=type_onemonth))
        ax.xaxis.set_minor_locator(mdates.YearLocator(1,month=type_onemonth))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.set_xlim(np.datetime64(td_str+'-'+str(type_onemonth).zfill(2)+'-01'),np.datetime64('2023-'+str(type_onemonth).zfill(2)+'-01'))
    if ylim:
        ax.set_ylim(ylim)
    ax.set_ylabel(ylabelabs[var])
    ax.legend(fontsize=8, edgecolor='w', loc=legend_loc[var][t_domain])
    ax.set_title('')
    ax.grid(alpha=0.3)

    SaveFig(fig, graph_type_str+'.abs', date_str, var, note)

    plt.close(fig)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


### Make double sea ice plots

In [23]:
%%time

if plot_types['line'] and double_plot:
    # Setup figure
    fig,axlist = plt.subplots(2,1, layout='constrained')
    fig.set_size_inches(wdth[t_domain],7)
    
    months = [3,9]
    let_list = ['(a)', '(b)']
    
    for mi in range(0,len(months)):
        m = months[mi]
        
        for dsname, attrs in ds_names.items():
            if attrs[0]:
                da_abs = ds_abs[dsname].loc[dict(month=m)]
                da_abs = da_abs.where(da_abs.time.dt.month.isin(m), drop=True)
    
                # LENS ensemble
                if attrs[1] == 0:
                    # Extract mean & min and max of envelope
                    da_abs_mean = ds_abs[dsname+' mean'].loc[dict(month=m)]
                    da_abs_mean = da_abs_mean.where(da_abs_mean.time.dt.month.isin(m), drop=True)
                    da_abs_min = ds_abs[dsname+' min'].loc[dict(month=m)]
                    da_abs_min = da_abs_min.where(da_abs_min.time.dt.month.isin(m), drop=True)
                    da_abs_max = ds_abs[dsname+' max'].loc[dict(month=m)]
                    da_abs_max = da_abs_max.where(da_abs_max.time.dt.month.isin(m), drop=True)
                    
                    # Plot envelope
                    axlist[mi].fill_between(da_abs_min[period].values, da_abs_min.values,
                        da_abs_max.values, color=attrs[2], alpha=0.1,
                        ec=None,label=ds_paper_names[dsname], zorder=attrs[4])
                    
                    # Plot mean
                    # da_abs_mean.plot(
                    #     ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4],label=ds_paper_names[dsname]+' mean')
    
                # PiC_UVnudge ensemble
                elif attrs[1] == 1 and not ens_type:
                    da_abs_r = ds_abs_r[dsname].loc[dict(month=m)]
                    index = dict(ensemble_member=1)
                    addon = ' ($\it{r}^2$='+'{:.2f})'.format(da_abs_r.values**2)
                        
                    # Plot first ensemble member
                    da_abs.loc[index].plot(
                        ax=axlist[mi], color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname]+addon)
    
                    # Plot other ensemble members
                    for i in range(2,4):
                        index['ensemble_member'] = i
                
                        da_abs.loc[index].plot(
                            ax=axlist[mi], color=attrs[2], x=period, zorder=attrs[4], ls=attrs[3])
                        
                # PiC_UVnudge single run & Observations (& PiC_UVensemble means)
                elif attrs[1] >= 2 or (ens_type and attrs[1] == 1):
                    addon = ' ($\it{r}^2$='+'{:.2f})'.format(ds_abs_r[dsname].loc[dict(month=m)].values**2) if attrs[1] < 3 else ''
                    
                    # Plot absolute value
                    da_abs.plot(
                        ax=axlist[mi], color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname]+addon)
            

        # Formatting
        if mi == 1:
            axlist[mi].set_xlabel(xlabel)
        else:
            axlist[mi].set_xlabel('')
        axlist[mi].xaxis.set_major_locator(mdates.YearLocator(5,month=m))
        axlist[mi].xaxis.set_minor_locator(mdates.YearLocator(1,month=m))
        axlist[mi].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        axlist[mi].set_xlim(np.datetime64(td_str+'-'+str(m).zfill(2)+'-01'),np.datetime64('2023-'+str(m).zfill(2)+'-01'))
        axlist[mi].set_ylabel(ylabelabs[var][m])
        axlist[mi].set_ylim(ylimabs[var][m])
        axlist[mi].legend(fontsize=8, edgecolor='w', loc=legend_loc[var][t_domain])
        axlist[mi].set_title('')
        axlist[mi].grid(alpha=0.3)
        axlist[mi].annotate(let_list[mi],(0.1,0.1),xytext=(-10,0),ha='right',va='center',
                    size=12,xycoords='axes fraction',textcoords='offset points')

    SaveFig(fig, graph_type_str+'.abs', date_str, var, note)

    plt.close(fig)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 4.77 µs


### Make number of nudged years plots

In [24]:
%%time

if plot_types['line'] and t_domain == 1950:
    # Setup figure
    fig,ax = plt.subplots(1,1, layout='constrained')
    fig.set_size_inches(wdth[t_domain],3)

    for dsname, attrs in ds_names.items():
        if attrs[0] and (dsname in ds_nudyr_names.keys()):
            da_nudyr = ds_nudyr[dsname]
            da_nudyr = da_nudyr.dropna(dim=period)
            da_trdline = ds_trdline[dsname]

            # PiC_UVnudge ensemble
            if attrs[1] == 1 and not ens_type:
                index = dict(ensemble_member=1)

                # Plot first ensemble member
                da_nudyr.loc[index].plot(
                    ax=ax, color=attrs[2], ls=attrs[3], lw=0.7, x=period, zorder=attrs[4], label=ds_nudyr_names[dsname], alpha=0.2)

                # Plot vertical line
                ax.axvline(da_nudyr[period].values[0], color='k', zorder=1)
                
                # Plot other ensemble members
                for i in range(2,4):
                    index['ensemble_member'] = i
            
                    da_nudyr.loc[index].plot(
                        ax=ax, color=attrs[2], x=period, lw=0.7, zorder=attrs[4], ls=attrs[3], alpha=0.2)

                    
            # PiC_UVnudge single run & Observations (& PiC_UVensemble means)
            elif attrs[1] == 2 or (ens_type and attrs[1] == 1):
                
                # Plot absolute value
                da_nudyr.plot(
                    ax=ax, color=attrs[2], ls=attrs[3], lw=0.7, x=period, zorder=attrs[4], label=ds_nudyr_names[dsname], alpha=0.3)

                ax.axvline(da_nudyr[period].values[0], color='k', zorder=1)

            # Plot trendline
            da_trdline.plot(
                ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4]+1, label=ds_nudyr_names[dsname]+' trend line')
            

    # Formatting
    ax.set_xlabel(xlabelnud)
    ax.set_xlim([0, 180])
    ax.set_xticks(ticks=np.arange(0,181,10), labels=np.arange(0,181,10))
    ax.set_ylabel(ylabelabs[var])
    ax.legend(edgecolor='w', loc='lower right', fontsize=8)
    ax.set_title('')
    ax.grid(alpha=0.3)

    SaveFig(fig, graph_type_str+'.nudyr', date_str, var, note)

    plt.close(fig)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


### Make anomaly plots

In [25]:
%%time

if plots['ts'][0]:
    # Setup figure
    fig,ax = plt.subplots(1,1, layout='constrained')
    fig.set_size_inches(10,5)

    for dsname, attrs in ds_names.items():
        if attrs[0] and (' drift' not in dsname):
            da_anom = ds_anom[dsname]

            # LENS ensemble
            if attrs[1] == 0:
                # Extract mean & min and max of envelople
                da_anom_mean = ds_anom[dsname+' mean']
                da_anom_min = ds_anom[dsname+' min']
                da_anom_max = ds_anom[dsname+' max']
                
                # Plot envelope
                ax.fill_between(da_anom_min[period].values, da_anom_min.values,
                    da_anom_max.values, color=attrs[2], alpha=0.1,
                    ec=None,label=dsname, zorder=attrs[4])
                
                # Plot mean
                da_anom_mean.plot(
                    ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4],label=dsname+' mean')

            # PiC_UVnudge ensemble
            elif attrs[1] == 1 and not ens_type:
                index = dict(ensemble_member=1)

                # Plot first ensemble member
                da_anom.loc[index].plot(
                    ax=ax, color=attrs[2], ls=attrs[3], x=period,
                    zorder=attrs[4], label=dsname+' ($\it{r}$='+'{:.2f})'.format(ds_anom_r[dsname].values))

                # Plot other ensemble members
                for i in range(2,4):
                    index['ensemble_member'] = i
            
                    da_anom.loc[index].plot(
                        ax=ax, color=attrs[2], x=period, zorder=attrs[4], ls=attrs[3])
                    
            # PiC_UVnudge single run & Observations (& PiC_UVensemble means)
            elif attrs[1] >= 2 or (ens_type and attrs[1] == 1):
                addon = '' if attrs[1] == 3 else ' ($\it{r}$='+'{:.2f})'.format(ds_anom_r[dsname].values)
                # Plot anomalies
                da_anom.plot(
                    ax=ax, color=attrs[2], ls=attrs[3], x=period,
                    zorder=attrs[4], label=dsname+addon)
            

    # Formatting
    ax.set_xlabel(xlabel)
    if time_avg == 1:
        ax.set_xlim(xlim)
        ax.set_xticks(ticks=xtick_loc,labels=xtick_lbl)
        ax.set_xticks(ticks=xtick_loc_min,minor=True)
    elif time_avg == 4:
        ax.xaxis.set_major_locator(mdates.YearLocator(5,month=1))
        ax.xaxis.set_minor_locator(mdates.YearLocator(1,month=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.set_ylabel(ylabelanom[var])
    ax.legend()
    ax.set_title('')
    ax.grid(alpha=0.3)

    SaveFig(fig, graph_type_str+'.anom', date_str, var, note)

    plt.close(fig)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


### Make detrended plots

In [26]:
%%time

if plots['ts'][0] and time_avg == 1:
    # Setup figure
    fig,ax = plt.subplots(1,1, layout='constrained')
    fig.set_size_inches(10,5)

    for dsname, attrs in ds_names.items():
        if attrs[0] and (' drift' not in dsname):
            da_dtrd = ds_dtrd[dsname]

            # LENS ensemble
            if attrs[1] == 0:
                # Extract mean & min and max of envelople
                da_dtrd_mean = ds_dtrd[dsname+' mean']
                da_dtrd_min = ds_dtrd[dsname+' min']
                da_dtrd_max = ds_dtrd[dsname+' max']
                
                # Plot envelope
                ax.fill_between(da_dtrd_min[period].values, da_dtrd_min.values,
                    da_dtrd_max.values, color=attrs[2], alpha=0.1,
                    ec=None,label=dsname, zorder=attrs[4])
                
                # Plot mean
                da_dtrd_mean.plot(
                    ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4],label=dsname+' mean')

            # PiC_UVnudge ensemble
            elif attrs[1] == 1 and not ens_type:
                index = dict(ensemble_member=1)

                # Plot first ensemble member
                da_dtrd.loc[index].plot(
                    ax=ax, color=attrs[2], ls=attrs[3], x=period,
                    zorder=attrs[4], label=dsname+' ($\it{r}$='+'{:.2f})'.format(ds_dtrd_r[dsname].values))

                # Plot other ensemble members
                for i in range(2,4):
                    index['ensemble_member'] = i
            
                    da_dtrd.loc[index].plot(
                        ax=ax, color=attrs[2], x=period, zorder=attrs[4], ls=attrs[3])
                    
            # PiC_UVnudge single run & Observations (& PiC_UVensemble means)
            elif attrs[1] >= 2 or (ens_type and attrs[1] == 1):
                addon = '' if attrs[1] == 3 else ' ($\it{r}$='+'{:.2f})'.format(ds_dtrd_r[dsname].values)
                # Plot anomalies
                da_dtrd.plot(
                    ax=ax, color=attrs[2], ls=attrs[3], x=period,
                    zorder=attrs[4], label=dsname+addon)
            

    # Formatting
    ax.set_xlabel(xlabel)
    ax.set_xlim(xlim)
    ax.set_xticks(ticks=xtick_loc,labels=xtick_lbl)
    ax.set_xticks(ticks=xtick_loc_min,minor=True)
    ax.set_ylabel(ylabeldtrd[var])
    ax.legend()
    ax.set_title('')
    ax.grid(alpha=0.3)

    SaveFig(fig, graph_type_str+'.dtrdanom', date_str, var, note)

    plt.close(fig)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.48 µs


### Make ensemble spread plots

In [27]:
%%time

if plots['ts'][0] and not ens_type:
    # Setup figure
    fig,ax = plt.subplots(1,1, layout='constrained')
    fig.set_size_inches(10,5)

    for dsname, attrs in ds_names.items():
        if attrs[0] and attrs[1] == 1:
            da_enspd = ds_enspd[dsname]

            # Plot ensemble spread
            da_enspd.plot(
                ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4],label=dsname)


    # Formatting
    ax.set_xlabel(xlabel)
    ax.set_xlim(xlim)
    ax.set_xticks(ticks=xtick_loc,labels=xtick_lbl)
    ax.set_xticks(ticks=xtick_loc_min,minor=True)
    ax.set_ylabel(ylabelensp[var])
    ax.legend()
    ax.set_title('')
    ax.grid(alpha=0.3)

    SaveFig(fig, graph_type_str+'.espd', date_str, var, note)

    plt.close(fig)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 4.77 µs


In [28]:
varnum = 0
for dsname, attrs in ds_names.items():
    varnum = varnum+1 if attrs[0] else varnum

## Spatial trend plots

### Set up

In [29]:
if plot_types['spatial']:
    if s_domain == 1:
        proj = ccrs.NorthPolarStereo()
        extent = [-180, 180, 50, 90]
    elif s_domain == 2:
        proj = ccrs.NorthPolarStereo()
        extent = [-180, 180, 20, 50]

    # Monthly
    if time_avg == 0:
        date_str = mon_str
        period = 'time'

    # Seasonally
    elif time_avg == 2:
        date_str = seas_str
        period = 'time'
    
    # Latitude, longitude, and level names (ERA5 and the CESM produced datasets have different names)
    xname = defaultdict(lambda: 'lon')
    xname['ERA5'] = 'lonE'
    yname = defaultdict(lambda: 'lat')
    yname['ERA5'] = 'latE'
    zname = defaultdict(lambda: 'plev')

    # Determine width of plots
    if ds_names['LENS2 piControl'][0]:
        wdth = int(np.ceil(varnum/2))
        hgt = 2
    else:
        wdth = 4
        hgt = varnum

    # Plot letter
    plot_let_full = np.array(['(a)', '(b)', '(c)', '(d)',
                              '(e)', '(f)', '(g)', '(h)',
                              '(i)', '(j)', '(k)', '(l)',
                              '(m)', '(n)', '(o)', '(p)'])
    plot_let = plot_let_full[0:(wdth*hgt)].reshape((hgt, wdth))

    if ds_names['LENS2 piControl'][0]:
        plot_let = {'ERA5': plot_let[0,0],
                         'LENS2 piControl mean': plot_let[1,0],
                         'PiC_UVnudge': plot_let[0,1],
                         'PiC_UVnudge_2006': plot_let[1,1],
                         'PiC_UVnudge_LM': plot_let[0,2],
                         'PiC_UVnudge_LM2006': plot_let[1,2],
                         'PiC_UVnudge_MM': plot_let[0,3],
                         'PiC_UVnudge_MM2006': plot_let[1,3]}

    # Do we add land features
    addland = (var == 'aice')

In [30]:
if plots['strd'][0]:
    graph_type_str = 'Map.Trend'

    if time_avg == 0:
        index_list = date_str
    elif time_avg == 2:
        index_list = date_str

    sbpt_shp = (hgt,wdth)
    figsz = (2*wdth,2*hgt)

    # Plot levels
    aice_levels = np.arange(-30.0, 31.0, 2.0)
    aice_levels[15] = 0.01
    trdlevels = {'TREFHT': np.arange(-2.6,2.7,0.2),
                  'aice': aice_levels,
                  'Z3': np.arange(-40,41,2),
                  'U': np.arange(-1.5,1.6,0.1),
                  'V': np.arange(-1.50,1.51,0.1)}
    
    trdticks = {'TREFHT': np.arange(-2.5,2.6,0.5),
                'aice': np.arange(-30,31,10),
                'Z3': np.arange(-40,41,10),
                'U': np.array([-1.5,-1.0,-0.5,0.0,0.5,1.0,1.5]),
                'V': np.arange(-1.50,1.51,0.5)}
    
    # Plotting variables
    trdcmaps = {'TREFHT': cm.vik,
                'aice': SubCmap(mpl.colormaps['seismic_r'], trdlevels['aice'], 'mid', np.array([0,0,0,0])),
                'Z3': SubCmap(cm.vik, trdlevels['Z3'], 'mid', np.array([0,0,0,0])), 
                'U': SubCmap(cm.vik, trdlevels['U'], 'mid', np.array([0,0,0,0])),
                'V': SubCmap(cm.vik, trdlevels['V'], 'mid', np.array([0,0,0,0]))}
    trdlabels = {'TREFHT': '2m air temperature trend (K/decade)',
                 'aice': 'Sea ice concentration trend (%/decade)',
                 'Z3': 'Geopotential height trend (m/decade)',
                  'U': 'Zonal wind trend (m s$^{-1}$/decade)',
                  'V': 'Meridional wind trend (m s$^{-1}$/decade)'}
    trdlabels1row = {'TREFHT': '2m air temperature\ntrend (K/decade)',
                 'aice': 'Sea ice concentration\ntrend (%/decade)',
                 'Z3': 'Geopotential height\ntrend (m/decade)',
                  'U': 'Zonal wind\ntrend (m s$^{-1}$/decade)',
                  'V': 'Meridional wind\ntrend (m s$^{-1}$/decade)'}

### Data loading

In [31]:
%%time

if plots['strd'][0]:
    if s_domain == 1:
        # Load spatial trends
        ds_sptrend = LoadData(graph_type_str, var, time_outstr)
        ds_sptrend = ds_sptrend*100 if var == 'aice' else ds_sptrend
        
        ds_sppval = LoadData(graph_type_str+'.pval', var, time_outstr)
    
        ds_sppcorr = LoadData(graph_type_str+'.pcorr', var, time_outstr)

    if var == 'Z3':
        ds_usptrend = LoadData(graph_type_str, 'U', time_outstr)
        ds_usppval = LoadData(graph_type_str+'.pval', 'U', time_outstr)

        ds_vsptrend = LoadData(graph_type_str, 'V', time_outstr)
        ds_vsppval = LoadData(graph_type_str+'.pval', 'V', time_outstr)      

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.48 µs


### Make plots
#### 2D plots with PI-control

In [32]:
%%time

if plots['strd'][0] and file_bool and ds_names['LENS2 piControl'][0]:
    # List of P-val variables
    pval_names = list(ds_sppval.keys())
    
    # Plotting of trends for 2m air temperature and sea ice
    for i in np.arange(0,len(index_list)):
        # Time index
        t_index = {time_outstr: index_list[i]}

        # Setup figure
        fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained', subplot_kw=dict(projection=proj))
        fig.set_size_inches(figsz[0],figsz[1])

        axes_loc = {'ERA5': axlist[0,0],
                    'LENS2 piControl mean': axlist[1,0],
                    'PiC_UVnudge': axlist[0,1],
                    'PiC_UVnudge_2006': axlist[1,1],
                    'PiC_UVnudge_LM': axlist[0,2],
                    'PiC_UVnudge_LM2006': axlist[1,2],
                    'PiC_UVnudge_MM': axlist[0,3],
                    'PiC_UVnudge_MM2006': axlist[1,3]}

        for dsname, attrs in ds_names.items():
            if attrs[0]:
                dsname = 'LENS2 piControl mean' if dsname == 'LENS2 piControl' else dsname
                da_trend = ds_sptrend[dsname].loc[t_index]

                # Plot data
                cax = da_trend.plot.contourf(
                        ax=axes_loc[dsname], x=xname[dsname], y=yname[dsname], cmap=trdcmaps[var], 
                        levels=trdlevels[var], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)
                

                # Plot p-vals
                if dsname in pval_names:
                    da_pval = ds_sppval[dsname].loc[t_index]
                    da_crit = ds_sppval[dsname+' pcrit'].loc[t_index].values
                    da_pval.plot.contourf(
                        ax=axes_loc[dsname],levels=[-0.01,da_crit,1],hatches=['...',None],colors='none',
                        add_colorbar=False,transform=ccrs.PlateCarree(),zorder=2)

                if 'PiC_UVnudge' in dsname:
                    da_corr = ds_sppcorr[dsname].loc[t_index]
                    addon = '\nr = {:.2f}'.format(da_corr.values)
                else:
                    addon = '\n'
                # Formatting
                if attrs[1] == 1: 
                    AxisLabels(axes_loc[dsname], plot_let[dsname]+' '+ds_paper_names[dsname]+' mean'+addon, addland)
                else:
                    AxisLabels(axes_loc[dsname], plot_let[dsname]+' '+ds_paper_names[dsname]+addon, addland)

        # Add colorbar
        cb = fig.colorbar(cax, ax=axlist[:,:], pad=0.1,shrink=0.75,fraction=0.1, extend='both')
        cb.set_label(label=trdlabels[var], fontsize=12)
        cb.ax.tick_params(labelsize=12)
        cb.set_ticks(ticks=trdticks[var])

        SaveFig(fig, graph_type_str, date_str[i], var, note)

        plt.close(fig)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.72 µs


#### 2D plots without PI-control

In [33]:
%%time

if plots['strd'][0] and file_bool and not ds_names['LENS2 piControl'][0]:
    # List of P-val variables
    pval_names = list(ds_sppval.keys())

    row_num = {'ERA5': 0,
               'PiC_UVnudge_2006': 1,
               'PiC_UVnudge_LM2006': 0,
               'PiC_UVnudge_MM2006': 1}
    
    if ds_names['PiC_UVnudge_MM2006'][0]:
        ds_names['ERA5'][0] = False
        figsz = (figsz[0],figsz[1]-2)
        sbpt_shp = (sbpt_shp[0]-1,sbpt_shp[1])

    # Setup figure
    fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained', subplot_kw=dict(projection=proj))
    fig.set_size_inches(figsz[0],figsz[1])
    
    # Plotting of trends for 2m air temperature and sea ice
    for i in np.arange(0,len(index_list)):
        # Time index
        t_index = {time_outstr: index_list[i]}

        for dsname, attrs in ds_names.items():
            if attrs[0]:
                da_trend = ds_sptrend[dsname].loc[t_index]

                # Plot data
                cax = da_trend.plot.contourf(
                        ax=axlist[row_num[dsname],i], cmap=trdcmaps[var], x=xname[dsname], y=yname[dsname],
                        levels=trdlevels[var], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)
                
                # Plot p-vals
                if dsname in pval_names:
                    da_pval = ds_sppval[dsname].loc[t_index]
                    da_crit = ds_sppval[dsname+' pcrit'].loc[t_index].values
                    da_pval.plot.contourf(
                        ax=axlist[row_num[dsname],i],levels=[-0.01,da_crit,1],hatches=['...',None],colors='none',
                        add_colorbar=False,transform=ccrs.PlateCarree(),zorder=2)

                # Formatting title & labels
                if 'PiC_UVnudge' in dsname:
                    da_corr = ds_sppcorr[dsname].loc[t_index]
                    addon = ' r = {:.2f}'.format(da_corr.values)
                else:
                    addon = ' '
                AxisLabels(axlist[row_num[dsname],i], plot_let[row_num[dsname], i]+addon, addland)
                
    
    # Add colorbar 
    cb = fig.colorbar(cax, ax=axlist[:,:], pad=0.1,shrink=0.75,fraction=0.1, extend='both')
    cb.set_label(label=trdlabels[var], fontsize=12)
    cb.ax.tick_params(labelsize=12)
    cb.set_ticks(ticks=trdticks[var])

    # Add row & column labels
    row_labels = [ds_paper_names[dsname] for dsname, attrs in ds_names.items() if attrs[0]]
    for ax, row in zip(axlist[:,0],row_labels):
        ax.annotate(row,(0,0.5),xytext=(-10,0),ha='right',va='center',
                    size=12,rotation=90,xycoords='axes fraction',textcoords='offset points')

    column_labels = date_str
    for ax, col in zip(axlist[0,:], column_labels):
        ax.annotate(col,(0.5,1),xytext=(0,25),ha='center',va='center',
                    size=12,xycoords='axes fraction',textcoords='offset points')

    SaveFig(fig, graph_type_str, time_outstr, var, note)

    plt.close(fig)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.25 µs


#### 3D plots without PI-control

In [34]:
%%time

# 3D variable plots
if plots['strd'][0] and not file_bool and not ds_names['LENS2 piControl'][0]:
    # List of P-val variables
    pval_names = list(ds_usppval.keys())
    row_labels = [ds_paper_names[dsname] for dsname, attrs in ds_names.items() if attrs[0]]

    if s_domain == 1:
        # Combine variables into dict
        trend_dict = {'Z3': [ds_sptrend, ds_sppval],
                      'U': [ds_usptrend, ds_usppval],
                      'V': [ds_vsptrend, ds_vsppval]}
        row_ind = {'ERA5': 0,
                   'PiC_UVnudge_2006': 1,
                   'PiC_UVnudge_LM2006': 2,
                   'PiC_UVnudge_MM2006': 3}
    else:
        trend_dict = {'U': [ds_usptrend, ds_usppval],
                      'V': [ds_vsptrend, ds_vsppval]}
        
    # Cycle through variables
    for v, ds_sets in trend_dict.items():
        dv_trend = ds_sets[0]
        dv_pval = ds_sets[1]
        
        # Cycle through plot levels
        for pl in plot_levels:
            dl_trend = dv_trend.loc[dict(plev=pl)]
            dl_pval = dv_pval.loc[dict(plev=pl)]
            
            # Setup figure
            fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained', subplot_kw=dict(projection=proj))
            fig.set_size_inches(figsz[0],figsz[1])

            if varnum == 1:
                axlist = np.expand_dims(axlist, axis=0)
            
            # Plotting of trends for geopotential height, zonal wind, and meridional wind
            for i in np.arange(0,len(index_list)):
                # Time index
                t_index = {time_outstr: index_list[i]}

                if varnum == 1:
                   axes_loc = {'ERA5': axlist[0, i]} 
                else:
                    axes_loc = {'ERA5': axlist[0, i],
                                'PiC_UVnudge_2006': axlist[1,i],
                                'PiC_UVnudge_LM2006': axlist[2,i],
                                'PiC_UVnudge_MM2006': axlist[3,i]}
        
                for dsname, attrs in ds_names.items():
                    if attrs[0]:
                        da_trend = dl_trend[dsname].loc[t_index]
        
                        # Plot data
                        # x=xname[dsname], y=yname[dsname],
                        cax = da_trend.plot.contourf(
                                ax=axes_loc[dsname],  cmap=trdcmaps[v], 
                                levels=trdlevels[v], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)
                        
                        # Plot p-vals
                        if dsname in pval_names:
                            da_pval = dl_pval[dsname].loc[t_index]
                            da_crit = dl_pval[dsname+' pcrit'].loc[t_index].values
                            da_pval.plot.contourf(
                                ax=axes_loc[dsname],levels=[-0.01,da_crit,1],hatches=['...',None],colors='none',
                                add_colorbar=False,transform=ccrs.PlateCarree(),zorder=2)
        
                        # Formatting title & labels
                        AxisLabels(axes_loc[dsname], plot_let[row_ind[dsname], i], addland)
                        

            if varnum == 1:
                cb = fig.colorbar(cax, ax=axlist[:,:], pad=0.1,shrink=0.9,fraction=0.15, extend='both')
                cb.set_label(label=trdlabels1row[v], fontsize=12)
                cb.ax.tick_params(labelsize=12)
                cb.set_ticks(ticks=trdticks[v])
               
            else:
                cb = fig.colorbar(cax, ax=axlist[:,:], pad=0.1,shrink=0.75,fraction=0.1, extend='both')
                cb.set_label(label=trdlabels[v], fontsize=12)
                cb.ax.tick_params(labelsize=12)
                cb.set_ticks(ticks=trdticks[v])

                # Add column labels
                column_labels = date_str
                for ax, col in zip(axlist[0,:], column_labels):
                    ax.annotate(col,(0.5,1),xytext=(0,25),ha='center',va='center',
                                size=12,xycoords='axes fraction',textcoords='offset points')
                
                # Add row labels
                for ax, row in zip(axlist[:,0],row_labels):
                    ax.annotate(row,(0,0.5),xytext=(-10,0),ha='right',va='center',
                                size=12,rotation=90,xycoords='axes fraction',textcoords='offset points')
        
            SaveFig(fig, graph_type_str, time_outstr, v, note, pl)
        
            plt.close(fig)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


#### 3D plots without PI-control and U/V only

In [35]:
%%time

# 3D variable plots - combine U&V into single plot
if plots['strd'][0] and not file_bool and not ds_names['LENS2 piControl'][0]:
    # List of P-val variables
    pval_names = list(ds_usppval.keys())
    row_labels = [ds_paper_names[dsname] for dsname, attrs in ds_names.items() if attrs[0]]

    plot_let = plot_let_full[0:(wdth*hgt*2)].reshape((hgt*2, wdth))

    if s_domain == 1:
        # Combine variables into dict
        trend_dict = {'U': [ds_usptrend, ds_usppval, 0],
                      'V': [ds_vsptrend, ds_vsppval, 1]}
        
    else:
        trend_dict = {'U': [ds_usptrend, ds_usppval],
                      'V': [ds_vsptrend, ds_vsppval]}
     

    # Cycle through plot levels
    for pl in plot_levels:
        # Setup figure
        fig, axlist = plt.subplots(sbpt_shp[0]*2, sbpt_shp[1], layout='constrained', subplot_kw=dict(projection=proj))
        fig.set_size_inches(figsz[0],figsz[1]*2)
        
        
        # Cycle through variables
        for v, ds_sets in trend_dict.items():
            dl_trend = ds_sets[0].loc[dict(plev=pl)]
            dl_pval = ds_sets[1].loc[dict(plev=pl)]
            axi = ds_sets[2]
            
            # Plotting of trends for geopotential height, zonal wind, and meridional wind
            for i in np.arange(0,len(index_list)):
                # Time index
                t_index = {time_outstr: index_list[i]}

                if varnum == 1:
                   axes_loc = {'ERA5': axlist[axi, i]} 
                else:
                    axes_loc = {'ERA5': axlist[0, i],
                                'PiC_UVnudge_2006': axlist[1,i],
                                'PiC_UVnudge_LM2006': axlist[1,i],
                                'PiC_UVnudge_MM2006': axlist[1,i]}
        
                for dsname, attrs in ds_names.items():
                    if attrs[0]:
                        da_trend = dl_trend[dsname].loc[t_index]
                        
                        # Plot data
                        # x=xname[dsname], y=yname[dsname],
                        cax = da_trend.plot.contourf(
                                ax=axes_loc[dsname],  cmap=trdcmaps[v], 
                                levels=trdlevels[v], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)
                    
                        # Plot p-vals
                        if dsname in pval_names:
                            da_pval = dl_pval[dsname].loc[t_index]
                            da_crit = dl_pval[dsname+' pcrit'].loc[t_index].values
                            da_pval.plot.contourf(
                                ax=axes_loc[dsname],levels=[-0.01,da_crit,1],hatches=['...',None],colors='none',
                                add_colorbar=False,transform=ccrs.PlateCarree(),zorder=2)
        
                        # Formatting title & labels
                        addon = ''
                        if 'PiC_UVnudge' in dsname:
                            AxisLabels(axes_loc[dsname], plot_let[1, i]+addon, addland)
                        else:
                            AxisLabels(axes_loc[dsname], plot_let[axi, i]+addon, addland)
                        

            

            if varnum == 1:
                cb = fig.colorbar(cax, ax=axlist[axi,:], pad=0.1,shrink=0.9,fraction=0.15, extend='both')
                cb.set_label(label=trdlabels1row[v], fontsize=12)
                cb.ax.tick_params(labelsize=12)
                cb.set_ticks(ticks=trdticks[v])
               
            else:
                cb = fig.colorbar(cax, ax=axlist[:,:], pad=0.1,shrink=0.75,fraction=0.1, extend='both')
                cb.set_label(label=trdlabels[v], fontsize=12)
                cb.ax.tick_params(labelsize=12)
                cb.set_ticks(ticks=trdticks[v])

                # Add row labels
                for ax, row in zip(axlist[:,0],row_labels):
                    ax.annotate(row,(0,0.5),xytext=(-10,0),ha='right',va='center',
                                size=12,rotation=90,xycoords='axes fraction',textcoords='offset points')

            # Add column labels
            column_labels = date_str
            for ax, col in zip(axlist[0,:], column_labels):
                ax.annotate(col,(0.5,1),xytext=(0,25),ha='center',va='center',
                            size=12,xycoords='axes fraction',textcoords='offset points')
                
                
        if varnum == 1:
            SaveFig(fig, graph_type_str, time_outstr, 'U-V', note, pl)
        else: 
            SaveFig(fig, graph_type_str, time_outstr, v, note, pl)
        
        plt.close(fig)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.96 µs


## Spatial plots
### Set up

In [36]:
if plots['map'][0]:
    graph_type_str = 'Map'

    if time_avg == 0:
        index_list = np.arange(1,13)
        index_name = 'month'
    elif time_avg == 2:
        index_list = date_str
        index_name = 'season'
    elif time_avg == 3:
        index_list = np.arange(1)
        index_name = ''
        
    figsz = (3.5*wdth,7)
    sbpt_shp = (2,wdth)

    # Plotting variables
    mcmaps = {'Z3': {'avg': cm.cork,
                     'var': cm.acton_r},
              'TREFHT': plt.cm.gist_rainbow_r,
              'aice': cm.devon}
    mlabels = {'Z3': {'avg':' (m)', 'var': ' variance (m$^2$)'},
               'TREFHT': '2m air temperature (K)',
               'aice': 'Sea ice concetration'}
    mlevels = {'Z3': {'avg':
                      {500: np.arange(-300,329,30), 
                      850: np.arange(-200,219,20), 
                      925: np.arange(-200,291,20)},
                      'var': 
                      {500: np.arange(0,3010,150), 
                      850: np.arange(0,3010,150), 
                      925: np.arange(0,3010,150)}
                     },   # Add to mean height
               'TREFHT': np.arange(220,301,2),
               'aice': np.array([0.0,0.05,0.10,0.15,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90,0.95,1.00])}
    mheight = {500: 5300,
               850: 1400,
               925: 700}

### Data loading

In [37]:
%%time

if plots['map'][0]:
    # Load spatial averages
    ds_sp = LoadData(graph_type_str, var, time_outstr)

    if var == 'Z3':
        ds_spv = LoadData(graph_type_str+'.var', var, time_outstr)
        ds_usp = LoadData(graph_type_str, 'U', time_outstr)
        ds_vsp = LoadData(graph_type_str, 'V', time_outstr)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


### Make plots

In [38]:
%%time

if plots['map'][0] and file_bool:
    # Plotting of spatial patterns for 2m air temperature and sea ice
    for i in np.arange(0,len(index_list)):
        # Time index
        t_index = {index_name:index_list[i]} if time_avg < 3 else dict()

        # Setup figure
        fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained', subplot_kw=dict(projection=proj))
        fig.set_size_inches(figsz[0],figsz[1])

        axes_loc = {'ERA5': axlist[0,0],
                    'LENS2 piControl mean': axlist[1,0],
                    'PiC_UVnudge': axlist[0,1],
                    'PiC_UVnudge_2006': axlist[1,1],
                    'PiC_UVnudge_LM': axlist[0,2],
                    'PiC_UVnudge_LM2006': axlist[1,2],
                    'PiC_UVnudge_MM': axlist[0,3],
                    'PiC_UVnudge_MM2006': axlist[1,3]}

        for dsname, attrs in ds_names.items():
            if attrs[0]:
                dsname = 'LENS2 piControl mean' if dsname == 'LENS2 piControl' else dsname
                da_sp = ds_sp[dsname].loc[t_index]

                # Plot data
                cax = da_sp.plot.contourf(
                        ax=axes_loc[dsname], x=xname[dsname], y=yname[dsname], cmap=mcmaps[var], 
                        levels=mlevels[var], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)

                # Formatting
                if attrs[1] == 1: 
                    AxisLabels(axes_loc[dsname], dsname+'mean', addland)
                else:
                    AxisLabels(axes_loc[dsname], dsname, addland)

        # Add colorbar
        cb = fig.colorbar(cax,ax=axlist[:,:],pad=0.1,shrink=0.75,fraction=0.1, extend='both')
        cb.set_label(label=mlabels[var], fontsize=12)
        cb.ax.tick_params(labelsize=12)

        SaveFig(fig, graph_type_str, date_str[i], var, note)

        plt.close(fig)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.72 µs


In [39]:
%%time

if plots['map'][0] and not file_bool:
    # Plotting of spatial pattern for U & V & geopotential height
    for i in np.arange(0,len(index_list)):
        for l in np.arange(0,len(plot_levels)):
    
            # Setup figure
            fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained', subplot_kw=dict(projection=proj))
            fig.set_size_inches(figsz[0],figsz[1])
    
            axes_loc = {'ERA5': axlist[0,0],
                    'LENS2 piControl mean': axlist[1,0],
                    'PiC_UVnudge': axlist[0,1],
                    'PiC_UVnudge_2006': axlist[1,1],
                    'PiC_UVnudge_LM': axlist[0,2],
                    'PiC_UVnudge_LM2006': axlist[1,2],
                    'PiC_UVnudge_MM': axlist[0,3],
                    'PiC_UVnudge_MM2006': axlist[1,3]}
    
            for dsname, attrs in ds_names.items():
                if attrs[0]:
                     # Time and level index
                    tl_index = {index_name:index_list[i], zname[dsname]: plot_levels[l]} if time_avg < 3 else {zname[dsname]: plot_levels[l]}
                    da_sp = ds_sp[dsname].loc[tl_index]
                    da_vsp = ds_vsp[dsname].loc[tl_index]
                    da_usp = ds_usp[dsname].loc[tl_index]


                    # Merge U & V datasets
                    da_uvsp = xr.merge([da_vsp.rename('V'), da_usp.rename('U')])
    
                    # Plot data
                    cax = da_sp.plot.contourf(
                            ax=axes_loc[dsname], x=xname[dsname], y=yname[dsname], cmap=mcmaps[var]['avg'], 
                            levels=mlevels[var]['avg'][plot_levels[l]]+mheight[plot_levels[l]], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)

                    qkey = da_uvsp.plot.quiver(
                            ax=axes_loc[dsname], x=xname[dsname], y=yname[dsname], u='U', v='V', regrid_shape=20,
                            zorder=2, transform=ccrs.PlateCarree(), add_guide=False)
    
                    # Formatting
                    if attrs[1] == 1: 
                        AxisLabels(axes_loc[dsname], dsname+'mean', False)
                    else:
                        AxisLabels(axes_loc[dsname], dsname, False)
    
            # Add colorbar
            cb = fig.colorbar(cax,ax=axlist[:],pad=0.1,shrink=0.75,fraction=0.1, extend='both')
            cb.set_label(label='Z'+str(plot_levels[l])+mlabels[var]['avg'], fontsize=12)
            cb.ax.tick_params(labelsize=12)

            # Add quiver key
            axlist[0,0].quiverkey(qkey, X=0.9, Y=0.9, U=0.5, label='0.5 m s$^{-1}$', labelpos='N',
                               fontproperties={'size':12})
            
            SaveFig(fig, graph_type_str, date_str[i], var, note, plot_levels[l])
    
            plt.close(fig)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [40]:
%%time

if plots['map'][0] and not file_bool:
    # Plotting of spatial pattern for U & V & geopotential height
    for i in np.arange(0,len(index_list)):
        for l in np.arange(0,len(plot_levels)):
    
            # Setup figure
            fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained', subplot_kw=dict(projection=proj))
            fig.set_size_inches(figsz[0],figsz[1])
    
            axes_loc = {'ERA5': axlist[0,0],
                    'LENS2 piControl mean': axlist[1,0],
                    'PiC_UVnudge': axlist[0,1],
                    'PiC_UVnudge_2006': axlist[1,1],
                    'PiC_UVnudge_LM': axlist[0,2],
                    'PiC_UVnudge_LM2006': axlist[1,2],
                    'PiC_UVnudge_MM': axlist[0,3],
                    'PiC_UVnudge_MM2006': axlist[1,3]}
    
            for dsname, attrs in ds_names.items():
                if attrs[0]:
                     # Time and level index
                    tl_index = {index_name:index_list[i], zname[dsname]: plot_levels[l]} if time_avg < 3 else {zname[dsname]: plot_levels[l]}
                    da_sp = ds_sp[dsname].loc[tl_index]
                    da_vsp = ds_vsp[dsname].loc[tl_index]
                    da_usp = ds_usp[dsname].loc[tl_index]


                    # Merge U & V datasets
                    da_uvsp = xr.merge([da_vsp.rename('V'), da_usp.rename('U')])
    
                    # Plot data
                    cax = da_sp.plot.contourf(
                            ax=axes_loc[dsname], x=xname[dsname], y=yname[dsname], cmap=mcmaps[var]['avg'], 
                            levels=mlevels[var]['avg'][plot_levels[l]]+mheight[plot_levels[l]], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)

                    qkey = da_uvsp.plot.quiver(
                            ax=axes_loc[dsname], x=xname[dsname], y=yname[dsname], u='U', v='V', regrid_shape=20,
                            zorder=2, transform=ccrs.PlateCarree(), add_guide=False)
    
                    # Formatting
                    if attrs[1] == 1: 
                        AxisLabels(axes_loc[dsname], dsname+'mean', False)
                    else:
                        AxisLabels(axes_loc[dsname], dsname, False)
    
            # Add colorbar
            cb = fig.colorbar(cax,ax=axlist[:],pad=0.1,shrink=0.75,fraction=0.1, extend='both')
            cb.set_label(label='Z'+str(plot_levels[l])+mlabels[var]['avg'], fontsize=12)
            cb.ax.tick_params(labelsize=12)

            # Add quiver key
            axlist[0,0].quiverkey(qkey, X=0.9, Y=0.9, U=0.5, label='0.5 m s$^{-1}$', labelpos='N',
                               fontproperties={'size':12})
            
            SaveFig(fig, graph_type_str, date_str[i], var, note, plot_levels[l])
    
            plt.close(fig)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


In [41]:
%%time

if plots['map'][0] and not file_bool:
    # Plotting of spatial pattern for U & V & variance of geopotential height
    for i in np.arange(0,len(index_list)):
        for l in np.arange(0,len(plot_levels)):
    
            # Setup figure
            fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained', subplot_kw=dict(projection=proj))
            fig.set_size_inches(figsz[0],figsz[1])
    
            axes_loc = {'ERA5': axlist[0,0],
                    'LENS2 piControl mean': axlist[1,0],
                    'PiC_UVnudge': axlist[0,1],
                    'PiC_UVnudge_2006': axlist[1,1],
                    'PiC_UVnudge_LM': axlist[0,2],
                    'PiC_UVnudge_LM2006': axlist[1,2],
                    'PiC_UVnudge_MM': axlist[0,3],
                    'PiC_UVnudge_MM2006': axlist[1,3]}
    
            for dsname, attrs in ds_names.items():
                if attrs[0]:
                     # Time and level index
                    tl_index = {index_name:index_list[i], zname[dsname]: plot_levels[l]} if time_avg < 3 else {zname[dsname]: plot_levels[l]}
                    da_spv = ds_spv[dsname].loc[tl_index]
                    da_vsp = ds_vsp[dsname].loc[tl_index]
                    da_usp = ds_usp[dsname].loc[tl_index]


                    # Merge U & V datasets
                    da_uvsp = xr.merge([da_vsp.rename('V'), da_usp.rename('U')])
    
                    # Plot data
                    cax = da_spv.plot.contourf(
                            ax=axes_loc[dsname], x=xname[dsname], y=yname[dsname], cmap=mcmaps[var]['var'], 
                            levels=mlevels[var]['var'][plot_levels[l]], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)

                    qkey = da_uvsp.plot.quiver(
                            ax=axes_loc[dsname], x=xname[dsname], y=yname[dsname], u='U', v='V', regrid_shape=20,
                            zorder=2, transform=ccrs.PlateCarree(), add_guide=False)
    
                    # Formatting
                    if attrs[1] == 1: 
                        AxisLabels(axes_loc[dsname], dsname+'mean', False)
                    else:
                        AxisLabels(axes_loc[dsname], dsname, False)
    
            # Add colorbar
            cb = fig.colorbar(cax,ax=axlist[:],pad=0.1,shrink=0.75,fraction=0.1, extend='both')
            cb.set_label(label='Z'+str(plot_levels[l])+mlabels[var]['var'], fontsize=12)
            cb.ax.tick_params(labelsize=12)

            # Add quiver key
            axlist[0,0].quiverkey(qkey, X=0.9, Y=0.9, U=0.5, label='0.5 m s$^{-1}$', labelpos='N',
                               fontproperties={'size':12})
            
            SaveFig(fig, graph_type_str+'.var', date_str[i], var, note, plot_levels[l])
    
            plt.close(fig)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


## Zonal trend plots
### Set up

In [42]:
if plot_types['zonal']:
    if s_domain == 1:
        xlim = [50, 90]
        latticks = np.arange(50,91,10)
    elif s_domain == 2:
        xlim = [20, 50]
        latticks = np.arange(20,51,10)

    # Monthly
    if time_avg == 0:
        date_str = mon_str
        period = 'time'
        index_list = date_str

    # Seasonally
    elif time_avg == 2:
        date_str = seas_str
        period = 'time'
        index_list = date_str
    
    # Latitude and level names (ERA5 and the CESM produced datasets have different names)
    yname = defaultdict(lambda: 'lat')
    yname['ERA5'] = 'latE'
    zname = defaultdict(lambda: 'plev')

    pticks = np.array([100,300,500,700,850,1000])
    plim = [1000, 100]

In [43]:
if plots['ztrd'][0]:
    graph_type_str = 'Zonal.Trend'

    # Determine width of plots
    wdth = 4
    hgt = 1 if varnum == 1 else 2

    sbpt_shp = (hgt,wdth)
    figsz = (2*wdth,2*hgt)

    # Plot letter
    plot_let_full = np.array(['(a)', '(b)', '(c)', '(d)',
                              '(e)', '(f)', '(g)', '(h)'])
    plot_let = plot_let_full[0:(wdth*hgt)].reshape((hgt, wdth))

    # Plot levels
    ztrdlevels = {'Z3': np.arange(-40,41,2),
                  'U': np.arange(-1.5,1.6,0.1),
                  'V': np.arange(-0.20,0.21,0.01)}

    ztrdcmaps = {'Z3': SubCmap(cm.vik, ztrdlevels['Z3'], 'mid', np.array([0,0,0,0])), 
                'U': SubCmap(cm.vik, ztrdlevels['U'], 'mid', np.array([0,0,0,0])),
                'V': SubCmap(cm.vik, ztrdlevels['V'], 'mid', np.array([0,0,0,0]))}

    ztrdticks = {'Z3': np.arange(-40,41,10),
                'U': np.array([-1.5,-1.0,-0.5,0.0,0.5,1.0,1.5]),
                'V': np.arange(-0.20,0.21,0.05)}

    # Plotting variables
    ztrdlabels = {'Z3': 'Geopotential height trend (m/decade)',
                  'U': 'Zonal wind trend (m s$^{-1}$/decade)',
                  'V': 'Meridional wind trend (m s$^{-1}$/decade)'}
    ztrdlabels1row = {'Z3': 'Geopotential height\ntrend (m/decade)',
                  'U': 'Zonal wind\ntrend (m s$^{-1}$/decade)',
                  'V': 'Meridional wind\ntrend (m s$^{-1}$/decade)'}

### Data loading

In [44]:
%%time

# Load zonal trends
if plots['ztrd'][0]:
    ds_uztrend = LoadData(graph_type_str, 'U', time_outstr)
    ds_vztrend = LoadData(graph_type_str, 'V', time_outstr)

    ds_uzpval = LoadData(graph_type_str+'.pval', 'U', time_outstr)
    ds_vzpval = LoadData(graph_type_str+'.pval', 'V', time_outstr)

    if s_domain == 1:
        ds_zpval = LoadData(graph_type_str+'.pval', var, time_outstr)
        ds_ztrend = LoadData(graph_type_str, var, time_outstr)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


### Make plots

In [45]:
%%time

if plots['ztrd'][0]:
    

    if s_domain == 1:
        # List of variables
        vards = {'Z3': [ds_ztrend, ds_zpval],
                 'U': [ds_uztrend, ds_uzpval],
                 'V': [ds_vztrend, ds_vzpval]}
    else:
        vards = {'U': [ds_uztrend, ds_uzpval],
                 'V': [ds_vztrend, ds_vzpval]}

    for v, ds_sets in vards.items():
        dv_ztrd = ds_sets[0]
        dv_zpval = ds_sets[1]

        pval_names = list(dv_zpval.keys())
        
        # Setup figure
        fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained')
        fig.set_size_inches(figsz[0],figsz[1])

        if varnum == 1:
            axlist = np.expand_dims(axlist, axis=0)
        
        for i in np.arange(0,len(index_list)):
            # Time index
            t_index = {time_outstr: index_list[i]}

            # Plotting zonal U & V winds
            if varnum == 1:
                axes_loc = {'ERA5': axlist[0, i]}
            else:
                axes_loc = {'ERA5': axlist[0, i],
                            'PiC_UVnudge_2006': axlist[1,i],
                            'PiC_UVnudge_LM2006': axlist[1,i],
                            'PiC_UVnudge_MM2006': axlist[1,i]}

            for dsname, attrs in ds_names.items():
                if attrs[0] and dsname != 'LENS2 piControl':
                    da_ztrd = dv_ztrd[dsname].loc[t_index]

                    # Plot data
                    cax = da_ztrd.plot.contourf(
                            ax=axes_loc[dsname], x=yname[dsname], y=zname[dsname], cmap=ztrdcmaps[v], yincrease=False,
                            levels=ztrdlevels[v], add_colorbar=False, add_labels=False, zorder=1)

                    if dsname in pval_names:
                            da_pval = dv_zpval[dsname].loc[t_index]
                            da_crit = dv_zpval[dsname+' pcrit'].loc[t_index].values
                            da_pval.plot.contourf(
                                ax=axes_loc[dsname],x='lat', y=zname[dsname],levels=[-0.01,da_crit,1],hatches=['...',None],colors='none',
                                add_colorbar=False,add_labels=False,zorder=2)

                    if dsname == 'ERA5':
                        title_str = plot_let[0,i]+' '+date_str[i]
                        #title_str = plot_let[0,i]+date_str[i]+', '+ds_paper_names[dsname]
                    else: 
                        title_str = plot_let[1,i]+' '+date_str[i]+', '+ds_paper_names[dsname]

                    # Adding difference plot
                    if i == 0:
                        # Labeling
                        axes_loc[dsname].set_ylabel('Pressure level (hPa)')
                        axes_loc[dsname].set_yticks(pticks)
            
                    else:
                        # Labeling
                        axes_loc[dsname].set_ylabel('')
                        axes_loc[dsname].set_yticks(pticks, labels=[])

                    axes_loc[dsname].set_title(title_str, loc='left',fontsize=10)
                    axes_loc[dsname].set_ylim(plim)
                    axes_loc[dsname].set_xlabel('Latitude')
                    axes_loc[dsname].set_xticks(latticks)

        if varnum == 1:
            # Add colorbars 
            cb = fig.colorbar(cax, ax=axlist[:,:], pad=0.05,fraction=0.2, extend='both')
            cb.set_label(label=ztrdlabels1row[v])
            #cb.ax.tick_params(labelsize=12)
            cb.set_ticks(ticks=ztrdticks[v])

        else:
            # Add colorbars 
            cb = fig.colorbar(cax, ax=axlist[:,:], pad=0.05,shrink=0.75,fraction=0.1, extend='both')
            cb.set_label(label=ztrdlabels[v])
            #cb.ax.tick_params(labelsize=12)
            cb.set_ticks(ticks=ztrdticks[v])

        SaveFig(fig, graph_type_str, time_outstr, v, note)

        plt.close(fig)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.01 µs


In [46]:
%%time

# Put U & V on one plot
if plots['ztrd'][0]:
    

    if s_domain == 1:
        # List of variables
        vards = {'U': [ds_uztrend, ds_uzpval, 0],
                 'V': [ds_vztrend, ds_vzpval, 1]}
    else:
        vards = {'U': [ds_uztrend, ds_uzpval],
                 'V': [ds_vztrend, ds_vzpval]}

    plot_let = plot_let_full[0:(wdth*hgt*2)].reshape((hgt*2, wdth))
    
    # Setup figure
    fig, axlist = plt.subplots(sbpt_shp[0]*2, sbpt_shp[1], layout='constrained')
    fig.set_size_inches(figsz[0],figsz[1]*2)
    
    for v, ds_sets in vards.items():
        dv_ztrd = ds_sets[0]
        dv_zpval = ds_sets[1]
        axi = ds_sets[2]

        pval_names = list(dv_zpval.keys())
        
        for i in np.arange(0,len(index_list)):
            # Time index
            t_index = {time_outstr: index_list[i]}

            # Plotting zonal U & V winds
            if varnum == 1:
                axes_loc = {'ERA5': axlist[axi, i]}
            else:
                axes_loc = {'ERA5': axlist[0, i],
                            'PiC_UVnudge_2006': axlist[1,i],
                            'PiC_UVnudge_LM2006': axlist[1,i],
                            'PiC_UVnudge_MM2006': axlist[1,i]}

            for dsname, attrs in ds_names.items():
                if attrs[0] and dsname != 'LENS2 piControl':
                    da_ztrd = dv_ztrd[dsname].loc[t_index]

                    # Plot data
                    cax = da_ztrd.plot.contourf(
                            ax=axes_loc[dsname], x=yname[dsname], y=zname[dsname], cmap=ztrdcmaps[v], yincrease=False,
                            levels=ztrdlevels[v], add_colorbar=False, add_labels=False, zorder=1)

                    if dsname in pval_names:
                            da_pval = dv_zpval[dsname].loc[t_index]
                            da_crit = dv_zpval[dsname+' pcrit'].loc[t_index].values
                            da_pval.plot.contourf(
                                ax=axes_loc[dsname],x='lat', y=zname[dsname],levels=[-0.01,da_crit,1],hatches=['...',None],colors='none',
                                add_colorbar=False,add_labels=False,zorder=2)

                    if dsname == 'ERA5':
                        title_str = plot_let[axi,i]
                        #title_str = plot_let[0,i]+date_str[i]+', '+ds_paper_names[dsname]
                    else: 
                        title_str = plot_let[1,i]+' '+date_str[i]+', '+ds_paper_names[dsname]

                    # Adding difference plot
                    if i == 0:
                        # Labeling
                        axes_loc[dsname].set_ylabel('Pressure level (hPa)')
                        axes_loc[dsname].set_yticks(pticks)
            
                    else:
                        # Labeling
                        axes_loc[dsname].set_ylabel('')
                        axes_loc[dsname].set_yticks(pticks, labels=[])

                    if axi == 0:
                        axes_loc[dsname].set_xlabel('')
                    else: 
                        axes_loc[dsname].set_xlabel('Latitude')
                        
                    axes_loc[dsname].set_title(title_str, loc='left',fontsize=10)
                    axes_loc[dsname].set_ylim(plim)
                    
                    axes_loc[dsname].set_xticks(latticks)

        if varnum == 1:
            # Add colorbars 
            cb = fig.colorbar(cax, ax=axlist[axi,:], pad=0.05,fraction=0.2, extend='both')
            cb.set_label(label=ztrdlabels1row[v])
            #cb.ax.tick_params(labelsize=12)
            cb.set_ticks(ticks=ztrdticks[v])

        else:
            # Add colorbars 
            cb = fig.colorbar(cax, ax=axlist[:,:], pad=0.05,shrink=0.75,fraction=0.1, extend='both')
            cb.set_label(label=ztrdlabels[v])
            #cb.ax.tick_params(labelsize=12)
            cb.set_ticks(ticks=ztrdticks[v])

    # Add column labels
    column_labels = date_str
    for ax, col in zip(axlist[0,:], column_labels):
        ax.annotate(col,(0.5,1),xytext=(0,25),ha='center',va='center',
                    size=12,xycoords='axes fraction',textcoords='offset points')

    SaveFig(fig, graph_type_str, time_outstr, 'U-V', note)

    plt.close(fig)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


## Zonal plots
### Set up

In [47]:
if plots['zon'][0]:
    graph_type_str = 'Zonal'

    # Determine width of plots
    wdth = 3
    hgt = 1

    # Plot letter
    plot_let_full = np.array(['(a)', '(b)', '(c)', '(d)'])
    plot_let = plot_let_full[0:(wdth*hgt)].reshape((hgt, wdth))

    
    plot_let = {'ERA5': plot_let[0,0],
                'PiC_UVnudge_2006': plot_let[0,1],
                'PiC_UVnudge_LM2006': plot_let[0,1],
                'PiC_UVnudge_MM2006': plot_let[0,1],
                'diff': plot_let[0,2]}

    sbpt_shp = (hgt,wdth)
    figsz = (3*wdth,2*hgt)

    # Plot levels
    zonlevels = {'U': np.arange(-30,31,2),
                  'V': np.arange(-2.5,2.6,0.25)}
    
    zdifflevels = {'U': np.array([-5, -2, -1, -0.5, 0, 0.5, 1, 2, 5]),
                  'V': np.array([-5, -2, -1, -0.5, 0, 0.5, 1, 2, 5])}
    
    zoncmaps = {'U': cm.cork,
               'V': cm.bam}
    
    zdiffcmaps = {'U': SubCmap(mpl.colormaps['seismic_r'], zdifflevels['U'], 'mid', np.array([0,0,0,0])),
                  'V': SubCmap(mpl.colormaps['seismic_r'], zdifflevels['V'], 'mid', np.array([0,0,0,0]))}
    
    zonticks = {'U': np.array([-30, -20, -10, 0, 10, 20, 30]),
                  'V': np.array([-2, -1, 0, 1.0, 2.0])}
    
    zdiffticks = {'U': np.array([-5, -2, -1, -0.5, 0, 0.5, 1, 2, 5]),
                  'V': np.array([-5, -2, -1, -0.5, 0, 0.5, 1, 2, 5])}
    
    # Plotting variables
    zonlabels = {'U': 'Zonal wind (m s$^{-1}$)',
                  'V': 'Meridional wind (m s$^{-1}$)'}
    
    zdifflabels = {'U': 'Zonal wind error (m s$^{-1}$)',
                    'V': 'Meridional wind error (m s$^{-1}$)'}

### Data loading

In [48]:
%%time

# Load zonal averages
if plots['zon'][0]:
    ds_uzon = LoadData(graph_type_str, 'U', time_outstr)
    ds_vzon = LoadData(graph_type_str, 'V', time_outstr)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.01 µs


### Make plots

In [49]:
%%time

if plots['zon'][0] and not file_bool:
    # List of wind variables
    varlist = np.array(['U', 'V'])
    for v in varlist:
        ds_zon = ds_uzon if v == 'U' else ds_vzon

        for i in np.arange(0,len(index_list)):
            # Time index
            t_index = {time_outstr: index_list[i]}

            # Setup figure
            fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained')
            fig.set_size_inches(figsz[0],figsz[1])
        
            # Plotting zonal U & V winds
            axes_loc = {'ERA5': axlist[0],
                        'PiC_UVnudge_2006': axlist[1],
                        'PiC_UVnudge_LM2006': axlist[1],
                        'PiC_UVnudge_MM2006': axlist[1]}

            for dsname, attrs in ds_names.items():
                if attrs[0] and dsname != 'LENS2 piControl':
                    da_zon = ds_zon[dsname].loc[t_index]

                    # Plot data
                    cax = da_zon.plot.contourf(
                            ax=axes_loc[dsname], x=yname[dsname], y=zname[dsname], cmap=zoncmaps[v], yincrease=False,
                            levels=zonlevels[v], add_colorbar=False, add_labels=False, zorder=1)

                    # Adding difference plot
                    if 'PiC_UVnudge' in dsname:
                        da_era = ds_zon['ERA5'].loc[t_index]
                        da_era = da_era.rename({'latE':'lat', 'levE':'plev'})

                        cax2 = (da_zon-da_era).plot.contourf(
                                ax=axlist[2], x=yname[dsname], y=zname[dsname], cmap=zdiffcmaps[v], yincrease=False,
                                levels=zdifflevels[v], add_colorbar=False, add_labels=False, zorder=1)

                        axlist[2].fill([60,60,90,90],[100,850,850,100],
                            facecolor='gray',edgecolor='black',alpha=.4,linestyle='--',zorder=2)

                        # Labeling
                        axes_loc[dsname].set_title(plot_let[dsname]+' '+ds_paper_names[dsname], loc='left', fontsize=10)
                        axes_loc[dsname].set_ylabel('')
                        axes_loc[dsname].set_yticks(pticks, labels=[])
                        axes_loc[dsname].set_ylim(plim)
                        axes_loc[dsname].set_xlabel('Latitude')
                        axes_loc[dsname].set_xticks(latticks)

                        axlist[2].set_title(plot_let['diff']+' '+ds_paper_names[dsname]+'–'+'ERA5', loc='left', fontsize=10)
                        axlist[2].set_ylabel('')
                        axlist[2].set_yticks(pticks, labels=[])
                        axlist[2].set_ylim(plim)
                        axlist[2].set_xlabel('Latitude')
                        axlist[2].set_xticks(latticks)

                    else:
                        # Labeling
                        axes_loc[dsname].set_title(plot_let[dsname]+' '+ds_paper_names[dsname], loc='left', fontsize=10)
                        axes_loc[dsname].set_ylabel('Pressure level (hPa)')
                        axes_loc[dsname].set_yticks(pticks)
                        axes_loc[dsname].set_ylim(plim)
                        axes_loc[dsname].set_xlabel('Latitude')
                        axes_loc[dsname].set_xticks(latticks)
        
            # Add colorbars 
            cb = fig.colorbar(cax, ax=axlist[:], shrink=1.25, pad=0.05,fraction=0.1, extend='both')
            cb.set_label(label=zonlabels[v])
            #cb.ax.tick_params(labelsize=12)
            cb.set_ticks(ticks=zonticks[v])

            cb2 = fig.colorbar(cax2, ax=axlist[:], shrink=1.25, pad=0.05,fraction=0.1, extend='both')
            cb2.set_label(label=zdifflabels[v])
            #cb2.ax.tick_params(labelsize=12)
            cb2.set_ticks(ticks=zdiffticks[v])

            # SaveFig(fig, graph_type_str+'.noNW', date_str[i], v, note)
            SaveFig(fig, graph_type_str, date_str[i], v, note)

            plt.close(fig)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs
